# Topic Modeling using LDA

In [78]:
import pandas as pd
import spacy
import en_core_web_sm
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim
import nltk
from nltk import FreqDist


### Read in the data

In [44]:
df = pd.read_json('Automotive_5.json', lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3F73SC1LY51OO,B00002243X,Alan Montgomery,"[4, 4]",I needed a set of jumper cables for my new car...,5,Work Well - Should Have Bought Longer Ones,1313539200,"08 17, 2011"
1,A20S66SKYXULG2,B00002243X,alphonse,"[1, 1]","These long cables work fine for my truck, but ...",4,Okay long cables,1315094400,"09 4, 2011"
2,A2I8LFSN2IS5EO,B00002243X,Chris,"[0, 0]",Can't comment much on these since they have no...,5,Looks and feels heavy Duty,1374710400,"07 25, 2013"
3,A3GT2EWQSO45ZG,B00002243X,DeusEx,"[19, 19]",I absolutley love Amazon!!! For the price of ...,5,Excellent choice for Jumper Cables!!!,1292889600,"12 21, 2010"
4,A3ESWJPAVRPWB4,B00002243X,E. Hernandez,"[0, 0]",I purchased the 12' feet long cable set and th...,5,"Excellent, High Quality Starter Cables",1341360000,"07 4, 2012"


Check how many reviews are there

In [45]:
print("The number of reviews in the file are :", df.shape[0])

The number of reviews in the file are : 20473


### Basic NLP analysis

Load the English NLP model from SpaCy

In [58]:
nlp = en_core_web_sm.load()

Extract all the tokens from each review, remove stopwords, lemmatize

In [60]:
text = df['reviewText'].tolist()

all_tokens = []

for doc in nlp.pipe(text):

    tokens = [token.lemma_ for token in doc if valid_token(token)]
    all_tokens.append(tokens)

In [69]:
all_tokens[-2:]

[['mask',
  'course',
  'describe',
  'half',
  'face',
  'long',
  'protection',
  'neck',
  'great',
  'cut',
  'wind',
  'jacket',
  'tuck'],
 ['good',
  'light',
  'weight',
  'cool',
  'night',
  'rain',
  'half',
  'helmet',
  'like',
  'wear',
  'light']]

### Topic Modeling using LDA (Latent Dirichlet Allocation)

Create a dictionary of every term in the corpus

In [70]:
dictionary = corpora.Dictionary(all_tokens)

Convert the list of tokenized reviews into a Document-Term-Matrix

In [73]:
doc_term_matrix = [dictionary.doc2bow(rev) for rev in all_tokens]

Create the LDA model

In [85]:
LDA = gensim.models.ldamodel.LdaModel

LDA_model = LDA(corpus = doc_term_matrix,
                id2word = dictionary,
                num_topics=5, 
                chunksize=1000,
                passes=50,
                random_state=100)

In [86]:
LDA_model.print_topics()

[(0,
  '0.067*"light" + 0.025*"bulb" + 0.019*"bright" + 0.014*"look" + 0.011*"led" + 0.009*"white" + 0.008*"instal" + 0.008*"color" + 0.007*"Jeep" + 0.007*"replace"'),
 (1,
  '0.026*"car" + 0.019*"product" + 0.016*"clean" + 0.015*"use" + 0.013*"towel" + 0.012*"good" + 0.012*"like" + 0.010*"work" + 0.010*"wash" + 0.009*"water"'),
 (2,
  '0.019*"battery" + 0.019*"car" + 0.012*"use" + 0.011*"power" + 0.011*"charge" + 0.010*"plug" + 0.010*"oil" + 0.009*"work" + 0.008*"device" + 0.008*"filter"'),
 (3,
  '0.012*"use" + 0.010*"work" + 0.009*"easy" + 0.009*"need" + 0.009*"small" + 0.009*"fit" + 0.007*"like" + 0.007*"place" + 0.007*"cover" + 0.007*"hold"'),
 (4,
  '0.016*"work" + 0.015*"blade" + 0.014*"good" + 0.012*"wiper" + 0.011*"buy" + 0.010*"price" + 0.010*"great" + 0.009*"tire" + 0.008*"easy" + 0.008*"like"')]

### Visualize the LDA model

In [87]:
pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(LDA_model, doc_term_matrix, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.094339 -0.010465       1        1  26.643026
4      0.065461  0.035165       2        1  24.903681
2      0.051368  0.191245       3        1  20.654181
3      0.096314 -0.192022       4        1  19.381399
0     -0.307482 -0.023923       5        1   8.417716, topic_info=         Term         Freq        Total Category  logprob  loglift
209     light  4711.000000  4711.000000  Default  30.0000  30.0000
49    battery  3005.000000  3005.000000  Default  29.0000  29.0000
4         car  9235.000000  9235.000000  Default  28.0000  28.0000
142     clean  3126.000000  3126.000000  Default  27.0000  27.0000
3659     bulb  1596.000000  1596.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
9       great   301.894196  5188.203613   Topic5  -5.3467  -0.3692
466      want   254.982086  2157.343018   Topic5  -5.5156   0.3394
44       work   272.970581  8239.529297   Topic5  -5.4474  -0.9325
55        buy   235.570786  3852.424316   Topic5  -5.5947  -0.3196
8        good   223.372696  6567.664062   Topic5  -5.6479  -0.9062

[325 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
13680      2  0.989398    ACDelco
112        1  0.074748     Amazon
112        2  0.847484     Amazon
112        3  0.076769     Amazon
12772      3  0.998605  Automatic
...      ...       ...        ...
372        2  0.461253       year
372        3  0.196324       year
372        5  0.022868       year
2117       1  0.258371     yellow
2117       5  0.739720     yellow

[496 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 3, 4, 1])